# Unprocessed dwelling information extractor
This notebook extracts usefull NaN information per dwelling and saves this into one file (csv & excel.  
It will this for both the 10s and hour sample rate dataframes.  
The unprocessed dataframes are loaded from: `//datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//`  
The nan information per dwelling is loaded from: `//datc//opschaler//nan_information//`  
The final product is saved in: `//datc//opschaler//dwelling_information//total_information//`  
It contains the following information per dwelling, for both the 10s and one hour sample rate:  
* dwelling id
* recorded days
* total NaN streaks 
* total NaN streaks > 2
* total NaNs [-]
* total NaNs [%]
* mean of NaNs
* median of NaNs
* std of NaNs
* first highest NaN streak (%)
* first highest NaN streak column
* second highest NaN streak (%)
* second highest NaN streak column
* third highest NaN streak (%)
* third highest NaN streak column

# Imports

In [1]:
import glob
import pandas as pd
import numpy as np

# Function definitions

In [2]:
def unprocessed_length_in_days(dwelling_id, type_):
    """
    Get the total amount of days of the unprocessed dwelling_id.
    """
    dir = '//datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//'
    df = pd.read_csv(dir+dwelling_id+'_'+type_+'.csv', delimiter='\t', parse_dates=['datetime'])
    columns = df.columns
    df = df['datetime'] # only keep the datetime column
    start_date = df[0]
    stop_date = df[-1:]
    
    del df # Free up memory
    total_days = (stop_date - start_date).dt.days # total amount of recorded days
    total_days = total_days.reset_index() # Messy way to convert the Series to a DataFrame

    return total_days['datetime'][0], columns


def nan_information_extractor(dwelling_id, path, type_):
    """
    Extracts usefull information from the nan info table from a dwelling id. 
    Output is a list with this information.
    """
    df = pd.read_csv(path, delimiter='\t')
    df = df.sort_values(by=['Amount of NaNs'], ascending=False) # Sort from highest to lowest amount of NaNs
    
    recorded_days, columns = unprocessed_length_in_days(dwelling_id, type_) # Length of unprocessed dataframe in days
    
    if df.empty: # If df is empty, return nothins
        print('Dataframe is empty: %s' % path)
        result = list(np.full(14, np.NaN)) # Make all outputs NaN
        result[0] = dwelling_id
        result[1] = recorded_days
        return result
    else:

        if type_ == 'hour':
            length = recorded_days*24
        elif type_ == '10s':
            length = recorded_days*24*60*6
        else: 
            print('type_ must be \'hour\' or \'10s')
    
        # Calculate usefull information
        total_gaps = len(df['Amount of NaNs'])
        total_gaps_larger_than_2 = len(df[df['Amount of NaNs'] > 2])
        total_nans = df['Amount of NaNs'].sum()
        total_nans_percentage = (total_nans / (length*len(columns)))*100
        mean = df['Amount of NaNs'].mean()
        median = df['Amount of NaNs'].median()
        std = df['Amount of NaNs'].std()
        
        # Try to get relevant values for the top 3 of NaN streaks
        # Problem with this is that often there are multiple columns which have the same NaN streak...
        try: 
            first_highest_p = (df['Amount of NaNs'][0]/ length)*100
            first_highest_column = df['Column name'][0]
        except:
            print('There is no 1st highest')
            first_highest_p = np.NaN
            first_highest_column = np.NaN
        
        try:
            second_highest_p = (df['Amount of NaNs'][1]/ length)*100
            second_highest_column = df['Column name'][1]
        except:
            print('There is no 2nd highest')
            second_highest_p = np.NaN
            second_highest_column = np.NaN
        
        try:
            third_highest_p = (df['Amount of NaNs'][3]/ length)*100
            third_highest_column = df['Column name'][3]
        except:
            print('There is no 3rd highest')
            third_highest_p = np.NaN
            third_highest_column = np.NaN
        
    
        # Put the results in a list
        result = [dwelling_id, recorded_days, total_gaps, total_gaps_larger_than_2, total_nans, total_nans_percentage, mean, median, 
                  std, first_highest_p, first_highest_column, second_highest_p, second_highest_column, third_highest_p, third_highest_column]
        
        return result

# Main

In [3]:
def main():
    nan_dir = '//datc//opschaler//nan_information//'
    paths_h = glob.glob(nan_dir+'*_hour.csv')
    ids_h = list(map(lambda x: x[-20:-9], paths_h))
    
    paths_s = glob.glob(nan_dir+'*_hour.csv')
    ids_s = list(map(lambda x: x[-20:-9], paths_h))
    
    results_h = []
    results_10s = []
    
    headers=['dwelling id', 'recorded days', 'total NaN streaks', 'total NaN streaks > 2','total NaNs [-]', 'total NaNs [%]', 'mean of NaNs', 'median of NaNs', 'std of NaNs', 
             'first highest NaN streak (%)', 'first highest NaN streak column', 
             'second highest NaN streak (%)', 'second highest NaN streak column', 
             'third highest NaN streak (%)', 'third highest NaN streak column']
    
    for i, path in enumerate(paths_h):
        dwelling_id = ids_h[i]
        type_ = 'hour'
        results_h.append(nan_information_extractor(dwelling_id, path, type_))
    
    for i, path in enumerate(paths_s):
        print('10s at %s of %s' % (i, len(paths_s)))
        dwelling_id = ids_s[i]
        type_ = '10s'
        results_10s.append(nan_information_extractor(dwelling_id, path, type_))
        
    # make df from list of lists, round all values within to 1 decimal.
    df_hour = pd.DataFrame.from_records(results_h, columns=headers).round(decimals=3) 
    df_10s = pd.DataFrame.from_records(results_10s, columns=headers).round(decimals=3) 
    
    # sort by recoded days, highest to lowest
    df_hour = df_hour.sort_values(by=['recorded days'], ascending=False) 
    df_10s = df_10s.sort_values(by=['recorded days'], ascending=False) 
    
    return df_10s, df_hour

# Run main() and save the result

In [4]:
%time info_hour, info_10s = main() # This takes ~2,5 minutes

Dataframe is empty: //datc//opschaler//nan_information/P01S01W7042_hour.csv
Dataframe is empty: //datc//opschaler//nan_information/P01S01W1554_hour.csv
There is no 3rd highest
Dataframe is empty: //datc//opschaler//nan_information/P01S01W0001_hour.csv
10s at 0 of 56
10s at 1 of 56
10s at 2 of 56
10s at 3 of 56
10s at 4 of 56
10s at 5 of 56
10s at 6 of 56
10s at 7 of 56
10s at 8 of 56
10s at 9 of 56
10s at 10 of 56
Dataframe is empty: //datc//opschaler//nan_information/P01S01W7042_hour.csv
10s at 11 of 56
10s at 12 of 56
10s at 13 of 56
10s at 14 of 56
10s at 15 of 56
Dataframe is empty: //datc//opschaler//nan_information/P01S01W1554_hour.csv
10s at 16 of 56
10s at 17 of 56
10s at 18 of 56
10s at 19 of 56
10s at 20 of 56
10s at 21 of 56
10s at 22 of 56
10s at 23 of 56
10s at 24 of 56
10s at 25 of 56
10s at 26 of 56
10s at 27 of 56
There is no 3rd highest
10s at 28 of 56
10s at 29 of 56
10s at 30 of 56
10s at 31 of 56
10s at 32 of 56
10s at 33 of 56
10s at 34 of 56
10s at 35 of 56
10s at

In [5]:
info_hour.to_csv('//datc//opschaler//dwelling_information//total_information//total_nan_information_hour.csv', sep='\t', index=False)
info_10s.to_csv('//datc//opschaler//dwelling_information//total_information//nan_information_10s.csv', sep='\t', index=False)

# Also save to Excel
writer = pd.ExcelWriter('//datc//opschaler//dwelling_information//total_information//total_nan_information.xlsx')
info_hour.to_excel(writer,'Hour dataframes', index=False)
info_10s.to_excel(writer,'10s dataframes', index=False)
writer.save()

print('FINISHED')

FINISHED
